In [1]:
import numpy as np
from learning import load_dataset
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

Using TensorFlow backend.


In [2]:
z, u, F = load_dataset("../datasets/dynamics.npz")
dt = 0.02
N = len(z)

[learning.py] loading dataset (../datasets/dynamics.npz)
[learning.py] dataset loaded (100000 samples)


In [3]:
z.shape, u.shape, F.shape

((100000, 11), (100000, 3), (100000, 11))

In [4]:
df = pd.DataFrame({"x": z[:, 0], "y": z[:, 1], "theta": z[:, 2], "v_x": z[:, 3], "v_y": z[:, 4], "omega": z[:, 5], "phi": z[:, 6], 
                   "w_fl": z[:, 7], "w_fr": z[:, 8], "w_rl": z[:, 9], "w_rr": z[:, 10],
                   "u_s": u[:, 0], "u_t": u[:, 1], "u_b": u[:, 2]})
df.head()

,x,y,theta,v_x,v_y,omega,phi,w_fl,w_fr,w_rl,w_rr,u_s,u_t,u_b
0,245.378708,29.820505,2.594295,-9.808808,4.369758,0.900334,0.407837,14.883656,17.806568,22.120200,24.595048,-0.739654,0.000000,0.241328
1,-113.956505,-16.039574,5.992599,54.845596,-17.194349,-1.032317,-0.136066,104.030568,102.378157,109.698098,109.242723,0.283473,0.483079,0.000000
2,-58.519550,88.148857,3.175248,-22.069576,-0.774465,0.187366,0.021317,35.166864,35.791824,82.811009,82.817595,-0.020927,0.798030,0.000000
3,-37.576122,134.544037,2.169974,-49.379143,72.733475,1.894747,0.184357,159.582618,162.385564,162.209935,165.524586,-0.261726,0.000000,0.000000
4,103.542381,-61.490860,5.990056,32.814468,-10.204882,-0.583393,-0.053368,59.651678,56.978539,71.682825,69.498520,0.052822,0.754628,0.000000


In [5]:
labels = pd.DataFrame()
labels["x''"] = (F[:, 3] - z[:, 3])/dt
labels["y''"] = (F[:, 4] - z[:, 4])/dt
labels["theta''"] = (F[:, 5] - z[:, 5])/dt

In [6]:
Y = labels["x''"]
X = df
X = sm.add_constant(X)

def vif(X):
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns

    return vif.round(1)

def fit(X, Y):
    reg = sm.OLS(Y, X).fit()
    return reg.summary()

fit(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    x''   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     285.8
Date:                Mon, 20 Apr 2020   Prob (F-statistic):               0.00
Time:                        13:25:40   Log-Likelihood:            -5.9466e+05
No. Observations:              100000   AIC:                         1.189e+06
Df Residuals:                   99985   BIC:                         1.190e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4844      0.914      7.096      0.000       4.693       8.275
x             -0.1135      0.003    -39.782      0.000      -0.119      -0.108
y             -0.0225      0.005     -4.694      0.000      -0.032      -0.013
theta         -0.4519      0.206     -2.191      0.028      -0.856      -0.048
v_x           -0.1110      0.010    -11.512      0.000      -0.130      -0.092
v_y           -0.2663      0.013    -20.258      0.000      -0.292      -0.241
omega         -2.5899      0.467     -5.543      0.000      -3.506      -1.674
phi            1.1994      2.521      0.476      0.634      -3.742       6.141
w_fl          -0.6730      0.145     -4.646      0.000      -0.957      -0.389
w_fr           0.5323      0.145      3.682      0.000       0.249       0.816
w_rl          -0.1746      0.124     -1.407      0.160      -0.418       0.069
w_rr           0.3474      0.124      2.804      0.005       0.105       0.590
u_s            5.4941      1.217      4.515      0.000       3.109       7.879
u_t           -5.5591      1.017     -5.467      0.000      -7.552      -3.566
u_b           -3.7614      1.808     -2.080      0.037      -7.305      -0.218
==============================================================================
Omnibus:                    28963.148   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3134423.459
Skew:                          -0.314   Prob(JB):                         0.00
Kurtosis:                      30.420   Cond. No.                     1.76e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
vif(X)

,VIF Factor,features
0,9.7,const
1,1.3,x
2,1.6,y
3,1.7,theta
4,1.5,v_x
5,1.9,v_y
6,5.7,omega
7,3.9,phi
8,652.6,w_fl
9,647.5,w_fr


In [8]:
X2 = X.drop(["w_fl", "w_rr", "w_rl", "omega", "phi", "u_b", "u_t", "const"], axis="columns")

fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    x''   R-squared (uncentered):                   0.037
Model:                            OLS   Adj. R-squared (uncentered):              0.037
Method:                 Least Squares   F-statistic:                              546.8
Date:                Mon, 20 Apr 2020   Prob (F-statistic):                        0.00
Time:                        13:25:42   Log-Likelihood:                     -5.9475e+05
No. Observations:              100000   AIC:                                  1.190e+06
Df Residuals:                   99993   BIC:                                  1.190e+06
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             -0.1084      0.003    -38.858      0.000      -0.114      -0.103
y             -0.0256      0.005     -5.359      0.000      -0.035      -0.016
theta          0.9018      0.131      6.887      0.000       0.645       1.158
v_x           -0.1126      0.010    -11.678      0.000      -0.131      -0.094
v_y           -0.2206      0.012    -18.218      0.000      -0.244      -0.197
w_fr           0.0361      0.005      7.089      0.000       0.026       0.046
u_s            4.8470      0.704      6.880      0.000       3.466       6.228
==============================================================================
Omnibus:                    28833.073   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3124834.187
Skew:                          -0.302   Prob(JB):                         0.00
Kurtosis:                      30.379   Cond. No.                         327.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
vif(X2)

,VIF Factor,features
0,1.5,x
1,1.6,y
2,2.6,theta
3,1.5,v_x
4,1.6,v_y
5,2.3,w_fr
6,1.0,u_s


In [10]:
X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
# X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["phi"])
X2 = X2.drop(["u_s"], axis="columns")


fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    x''   R-squared (uncentered):                   0.255
Model:                            OLS   Adj. R-squared (uncentered):              0.255
Method:                 Least Squares   F-statistic:                              3108.
Date:                Mon, 20 Apr 2020   Prob (F-statistic):                        0.00
Time:                        13:25:42   Log-Likelihood:                     -5.8192e+05
No. Observations:              100000   AIC:                                  1.164e+06
Df Residuals:                   99989   BIC:                                  1.164e+06
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
x              -0.0779      0.002    -31.189      0.000      -0.083      -0.073
y              -0.0128      0.004     -2.970      0.003      -0.021      -0.004
theta           0.7136      0.116      6.158      0.000       0.487       0.941
v_x            -0.5765      0.012    -50.027      0.000      -0.599      -0.554
v_y            -0.6140      0.014    -44.804      0.000      -0.641      -0.587
w_fr            0.0346      0.005      7.601      0.000       0.026       0.044
a_cos_theta    83.5500      0.939     89.013      0.000      81.710      85.390
b_cos_theta  -262.6511      2.053   -127.951      0.000    -266.674    -258.628
a_sin_theta    45.9328      1.016     45.221      0.000      43.942      47.924
b_sin_theta   -47.4559      2.163    -21.940      0.000     -51.695     -43.216
v_sin_phi      -0.3013      0.034     -8.766      0.000      -0.369      -0.234
==============================================================================
Omnibus:                    33974.544   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7535839.323
Skew:                          -0.394   Prob(JB):                         0.00
Kurtosis:                      45.520   Cond. No.                     1.14e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.14e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
vif(X2)

,VIF Factor,features
0,1.6,x
1,1.7,y
2,2.7,theta
3,2.8,v_x
4,2.7,v_y
5,2.4,w_fr
6,2.4,a_cos_theta
7,1.1,b_cos_theta
8,2.3,a_sin_theta
9,1.1,b_sin_theta


In [12]:
fit(X2[["a_cos_theta", "b_cos_theta", "const"]], Y)

KeyError: "['const'] not in index"

In [ ]:
import matplotlib.pyplot as plt

# plt.scatter(X, Y, s=1)
# plt.plot(X, reg.intercept_ + reg.coef_[0] * X)
# plt.show()


In [ ]:
Y = labels["y''"]
X = df.drop(["theta", "w_rr", "w_fl", "w_fr", "v_x"], axis="columns")
X = sm.add_constant(X)

fit(X, Y)

In [ ]:
vif(X)

In [ ]:
X2 = X.drop(["phi", "w_rl", "u_b"], axis="columns")

X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["phi"])

fit(X2, Y)

In [ ]:
vif(X2)

In [ ]:
Y = labels["theta''"]
X = df.drop(["x", "y", "w_rl", "w_fl", "w_fr"], axis="columns")
X = sm.add_constant(X)

fit(X, Y)

In [ ]:
vif(X)

In [ ]:
X2 = X.drop(["w_rr", "theta", "v_y"], axis="columns")

X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
# X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
# X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
# X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["phi"])

fit(X2, Y)

In [ ]:
vif(X2)